In [1]:
import pandas as pd
import os
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# Set path using Path.cwd()
path_cwd = Path.cwd()
path_data = str(path_cwd) + '/CSV/'
path_data_more = str(path_cwd) + '/CSV_more/'

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
import plotly.express as px
from pathlib import Path

# Set path
path_cwd = Path.cwd()
path_data_more = str(path_cwd) + '/data_input/'

# Files
files = ['2003-2005_CHS.csv', '2007-2014_CHS.csv', '2015-2022_CHS.csv']

# Indicator map (updated with missing variations)
indicator_map = {
    'Very good or excellent self-rated mental health': 'perceived mental health good',
    'Perceived mental health, very good or excellent': 'perceived mental health good',
    'Fair or poor self-rated mental health': 'perceived mental health poor',  # If exists in early data
    'Perceived mental health, fair or poor': 'perceived mental health poor',
    'Life satisfaction, satisfied or very satisfied': 'life satisfaction',
    'Quite a lot of life stress': 'life stress',
    'Perceived life stress, quite a lot': 'life stress',
    'Perceived life stress, most days quite a bit or extremely stressful': 'life stress',
    'Life stress, quite a lot (18 years and over)': 'life stress',
    'Very strong or somewhat strong sense of belonging to local community': 'sense of belonging',
    'Sense of belonging to local community, somewhat strong or very strong': 'sense of belonging',
    'Mood disorder': 'mood disorder',
    'Anxiety disorder': 'anxiety disorder'
}

# Char map (unchanged)
char_map = {
    'Number of persons': 'number',
    'Percent': 'percent',
    'Low 95% confidence interval, number of persons': 'low95_number',
    'High 95% confidence interval, number of persons': 'high95_number',
    'Low 95% confidence interval, percent': 'low95_percent',
    'High 95% confidence interval, percent': 'high95_percent',
    'Statistically different from the Canada rate': 'diffcan',
    'Statistically different from the provincial rate': 'diffprov',
    'Statistically different from the peer group rate': 'diffpeer',
    'Statistically different from previous cycle': 'diffprev',
    'Statistically different from previous reference period': 'diffprev'
}

# Geog map (unchanged)
geog_map = {
    'Canada': 'Canada',
    'Newfoundland and Labrador': 'Newfoundland and Labrador',
    'Prince Edward Island': 'Prince Edward Island',
    'Nova Scotia': 'Nova Scotia',
    'New Brunswick': 'New Brunswick',
    'Quebec': 'Quebec',
    'Ontario': 'Ontario',
    'Manitoba': 'Manitoba',
    'Saskatchewan': 'Saskatchewan',
    'Alberta': 'Alberta',
    'British Columbia': 'British Columbia',
    'Yukon': 'Yukon',
    'Northwest Territories': 'Northwest Territories',
    'Nunavut': 'Nunavut',
    'Calgary Zone, Alberta': 'Calgary',
    'Calgary Health Region, Alberta': 'Calgary',
    'Calgary, Alberta': 'Calgary',
    'Lethbridge, Alberta': 'Lethbridge',
    'Edmonton, Alberta': 'Edmonton',
    'South Zone, Alberta': 'South Zone',
    'Central Zone, Alberta': 'Central Zone',
    'North Zone, Alberta': 'North Zone',
    'Large urban population centres, Alberta': 'Large Urban Alberta',
    'Medium population centres, Alberta': 'Medium Alberta',
    'Small population centres, Alberta': 'Small Alberta',
    'Rural areas, Alberta': 'Rural Alberta',
    'Chinook Regional Health Authority, Alberta': 'Chinook',
    'Palliser Health Region, Alberta': 'Palliser',
    'David Thompson Regional Health Authority, Alberta': 'David Thompson',
    'East Central Health, Alberta': 'East Central',
    'Capital Health, Alberta': 'Capital',
    'Aspen Regional Health Authority, Alberta': 'Aspen',
    'Peace Country Health, Alberta': 'Peace Country',
    'Northern Lights Health Region, Alberta': 'Northern Lights'
}

# Data dict {geog: DF with 'period' index, 'indicator_char' columns}
data_dict = {}

# Process files
for file in files:
    df = pd.read_csv(path_data_more + file)
    
    # Standardize indicator and char
    df['indicator'] = df['Indicators'].map(indicator_map)
    df['char'] = df['Characteristics'].map(char_map)
    df = df.dropna(subset=['indicator', 'char'])
    
    # Map geog
    df['geog'] = df['GEO'].map(geog_map)
    df = df.dropna(subset=['geog'])
    
    # Pivot with aggfunc='first' for duplicates
    pivot = df.pivot_table(index=['geog', 'REF_DATE'], columns=['indicator', 'char'], values='VALUE', aggfunc='first')
    pivot.columns = ['_'.join(col) for col in pivot.columns]
    
    # Add to data_dict
    for g in pivot.index.get_level_values('geog').unique():
        g_df = pivot.xs(g).rename_axis('period')
        g_df.index = g_df.index.astype(str)  # Ensure str
        g_df = g_df.apply(pd.to_numeric, errors='coerce')
        if g not in data_dict:
            data_dict[g] = g_df
        else:
            data_dict[g] = data_dict[g].combine_first(g_df)

# Sort periods
for g in data_dict:
    data_dict[g] = data_dict[g].sort_index(key=lambda x: x.str.split('/').str[0].astype(int))

# Updated summarize function (with significance in trend)
def summarize_indicator(indicator, char='percent', regions=None):
    if not regions:
        regions = list(data_dict.keys())
    summaries = []
    for reg in regions:
        col = f'{indicator}_{char}'
        if reg in data_dict and col in data_dict[reg].columns:
            df = data_dict[reg][col].dropna()
            if df.empty:
                continue
            min_val = df.min()
            min_period = df.idxmin()
            max_val = df.max()
            max_period = df.idxmax()
            avg_val = df.mean()
            # Trend with significance
            periods_num = df.index.str.split('/').str[0].astype(int).tolist()
            if len(df) > 1:
                slope, _, _, p, _ = linregress(periods_num, df.values)
                trend = 'upward' if slope > 0 else 'downward' if slope < 0 else 'stable'
                sig = 'significant' if p < 0.05 else 'not significant'
                trend_str = f'{trend} ({sig})'
            else:
                trend_str = 'insufficient data'
            summaries.append({
                'region': reg,
                'min': min_val,
                'min_period': min_period,
                'max': max_val,
                'max_period': max_period,
                'average': avg_val,
                'trend': trend_str
            })
    if summaries:
        summary_df = pd.DataFrame(summaries)
        print(f"Summary for {indicator}:")
        print(summary_df.to_string(index=False))
        
        # Calgary-focused comparisons
        if 'Calgary' in summary_df['region'].values:
            cal_avg = summary_df[summary_df['region'] == 'Calgary']['average'].values[0]
            print("\nCalgary Comparisons:")
            for _, row in summary_df.iterrows():
                if row['region'] != 'Calgary':
                    comp = 'higher' if cal_avg > row['average'] else 'lower' if cal_avg < row['average'] else 'equal'
                    print(f"Calgary average ({cal_avg:.1f}%) is {comp} than {row['region']} ({row['average']:.1f}%).")
                    print(f"Trend in Calgary: {summary_df[summary_df['region'] == 'Calgary']['trend'].values[0]}.")
        
        return summary_df
    print(f"No data for {indicator}.")
    return None




In [3]:
#MAKE Values A LIST
geog_elements = list(geog_map.values()) #this is to use and compare ALL the regions 
#summarize and compare all regions for perceived mental health good
summarize_indicator('perceived mental health good', regions=geog_elements)


Summary for perceived mental health good:
                   region  min min_period  max max_period   average                      trend
                   Canada 56.9  2021/2022 74.6  2007/2008 70.020000     downward (significant)
Newfoundland and Labrador 58.3  2021/2022 77.0  2007/2008 71.510000     downward (significant)
     Prince Edward Island 55.9  2021/2022 75.3       2005 70.140000     downward (significant)
              Nova Scotia 54.6  2021/2022 73.6  2009/2010 67.500000     downward (significant)
            New Brunswick 54.6  2021/2022 70.8  2007/2008 65.550000     downward (significant)
                   Quebec 65.3  2021/2022 77.2  2007/2008 73.570000     downward (significant)
                  Ontario 54.6  2021/2022 71.1  2015/2016 64.875000 downward (not significant)
                 Manitoba 55.3  2021/2022 72.9       2003 68.070000     downward (significant)
             Saskatchewan 52.6  2021/2022 71.9  2007/2008 67.560000     downward (significant)
        

,region,min,min_period,max,max_period,average,trend
0,Canada,56.9,2021/2022,74.6,2007/2008,70.020000,downward (significant)
1,Newfoundland and Labrador,58.3,2021/2022,77.0,2007/2008,71.510000,downward (significant)
2,Prince Edward Island,55.9,2021/2022,75.3,2005,70.140000,downward (significant)
3,Nova Scotia,54.6,2021/2022,73.6,2009/2010,67.500000,downward (significant)
4,New Brunswick,54.6,2021/2022,70.8,2007/2008,65.550000,downward (significant)
5,Quebec,65.3,2021/2022,77.2,2007/2008,73.570000,downward (significant)
6,Ontario,54.6,2021/2022,71.1,2015/2016,64.875000,downward (not significant)
7,Manitoba,55.3,2021/2022,72.9,2003,68.070000,downward (significant)
8,Saskatchewan,52.6,2021/2022,71.9,2007/2008,67.560000,downward (significant)
9,Alberta,55.8,2021/2022,75.0,2007/2008,70.460000,downward (significant)


In [4]:
# Plot for Calgary
def plot_indicator(indicator, char='percent', regions=['Calgary']): #Change regions=[...] to add all the regions we want to plot to compare 
    dfs = []
    for reg in regions:
        col = f'{indicator}_{char}'
        if reg in data_dict and col in data_dict[reg].columns:
            df = data_dict[reg][col].reset_index().rename(columns={col: 'value'})
            df['region'] = reg
            dfs.append(df)
    if dfs:
        combined = pd.concat(dfs)
        fig = px.line(combined, x='period', y='value', color='region', title=f'{indicator.capitalize()} {char.capitalize()} Trend')
        fig.show()

# # Example plot
plot_indicator('perceived mental health good')


In [5]:
#Example usage: Summarize all indicators (add your regions list if needed)
indicators = ['perceived mental health good', 'perceived mental health poor', 'life satisfaction', 'life stress', 'sense of belonging']
for ind in indicators:
    summarize_indicator(ind)
    print("\n")

Summary for perceived mental health good:
                   region  min min_period  max max_period   average                      trend
                  Alberta 55.8  2021/2022 75.0  2007/2008 70.460000     downward (significant)
                    Aspen 66.7       2005 72.4  2009/2010 69.275000   upward (not significant)
         British Columbia 53.1  2021/2022 71.4  2007/2008 67.130000     downward (significant)
                  Calgary 57.7  2021/2022 77.8  2007/2008 72.410000     downward (significant)
                   Canada 56.9  2021/2022 74.6  2007/2008 70.020000     downward (significant)
                  Capital 72.1       2005 74.2  2007/2008 73.075000   upward (not significant)
                  Chinook 69.1       2003 74.5  2009/2010 72.725000   upward (not significant)
           David Thompson 69.4       2003 72.7  2007/2008 71.250000   upward (not significant)
             East Central 68.9  2009/2010 75.1  2007/2008 71.825000 downward (not significant)
        

In [6]:
# #Example: summarize life stress indicator
# print("indicator summarized: life stress")
# summarize_indicator('life stress', regions=['Calgary', 'Lethbridge', 'Alberta', 'Canada', 'Edmonton', 'Manitoba', 'Saskatchewan'])


In [7]:
#Example: summarize anxiety disorder indicator
print("indicator summarized: anxiety disorder")
summarize_indicator('anxiety disorder', regions=['Calgary', 'Lethbridge', 'Alberta', 'Canada', 'Edmonton', 'Manitoba', 'Saskatchewan'])

indicator summarized: anxiety disorder
Summary for anxiety disorder:
      region  min min_period  max max_period  average             trend
     Calgary 12.2  2021/2022 12.2  2021/2022     12.2 insufficient data
  Lethbridge 17.6  2021/2022 17.6  2021/2022     17.6 insufficient data
     Alberta 12.8  2021/2022 12.8  2021/2022     12.8 insufficient data
      Canada 12.7  2021/2022 12.7  2021/2022     12.7 insufficient data
    Edmonton 12.9  2021/2022 12.9  2021/2022     12.9 insufficient data
    Manitoba 11.6  2021/2022 11.6  2021/2022     11.6 insufficient data
Saskatchewan 13.1  2021/2022 13.1  2021/2022     13.1 insufficient data

Calgary Comparisons:
Calgary average (12.2%) is lower than Lethbridge (17.6%).
Trend in Calgary: insufficient data.
Calgary average (12.2%) is lower than Alberta (12.8%).
Trend in Calgary: insufficient data.
Calgary average (12.2%) is lower than Canada (12.7%).
Trend in Calgary: insufficient data.
Calgary average (12.2%) is lower than Edmonton (12.9%).

,region,min,min_period,max,max_period,average,trend
0,Calgary,12.2,2021/2022,12.2,2021/2022,12.2,insufficient data
1,Lethbridge,17.6,2021/2022,17.6,2021/2022,17.6,insufficient data
2,Alberta,12.8,2021/2022,12.8,2021/2022,12.8,insufficient data
3,Canada,12.7,2021/2022,12.7,2021/2022,12.7,insufficient data
4,Edmonton,12.9,2021/2022,12.9,2021/2022,12.9,insufficient data
5,Manitoba,11.6,2021/2022,11.6,2021/2022,11.6,insufficient data
6,Saskatchewan,13.1,2021/2022,13.1,2021/2022,13.1,insufficient data


In [8]:
# # Example plot (like your original, for perceived mental health good)
# regions_plot = ['Calgary', 'Alberta', 'Canada']  # Add more regions as needed
# data_plot = pd.DataFrame()
# for reg in regions_plot:
#     if reg in data_dict:
#         df_reg = data_dict[reg][['perceived mental health good_percent']].reset_index()
#         df_reg['region'] = reg
#         data_plot = pd.concat([data_plot, df_reg])

# if not data_plot.empty:
#     fig = px.bar(data_plot, x='period', y='perceived mental health good_percent', color='region', barmode='group',
#                  title='Perceived Mental Health Good (%) Over Time')
#     fig.update_layout(yaxis_title='Percentage (%)', xaxis_title='Period', legend_title='Region')
#     fig.show()
# else:
#     print("No plot data available.")

In [9]:
# List of indicators to graph (only perceived mental health good)
indicators = ['perceived mental health good']

# Colors for regions (customize as needed, e.g., to match report's orange theme)
region_colors = {'Calgary': '#FF8C00', 'Alberta': '#0000FF', 'Canada': '#008000'}

# Loop over each indicator (just one) and create a bar graph
for ind in indicators:
    # Collect data from each region
    dfs = []
    for reg in ['Calgary', 'Alberta', 'Canada']:
        if reg in data_dict and f'{ind}_percent' in data_dict[reg].columns:
            df = data_dict[reg][[f'{ind}_percent']].reset_index()
            df['region'] = reg
            df = df.rename(columns={f'{ind}_percent': 'value', 'period': 'period'})
            dfs.append(df)
    
    if dfs:
        combined_df = pd.concat(dfs)
        
        # Create grouped bar graph
        fig = px.bar(combined_df, x='period', y='value', color='region', barmode='group',
                     title='Perceived Mental Health Good (%) Over Time',
                     labels={'value': 'Percentage (%)', 'period': 'Period', 'region': 'Region'},
                     color_discrete_map=region_colors)
        
        fig.update_layout(xaxis={'categoryorder': 'array', 'categoryarray': combined_df['period'].unique()},  # Preserve order
                          yaxis_range=[0, 100],  # Assume 0-100% scale
                          height=500, width=800)
        fig.show()
    else:
        print(f"No data for {ind}")